In [1]:
import numpy as np
import torch
import datasets
import pandas as pd
# import editdistance
import transformers
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from accelerate import Accelerator
from sklearn.model_selection import StratifiedShuffleSplit
import os
from tqdm import tqdm
import re





===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/user/ksharma/miniconda3/envs/ks_tars_thesis did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/user/ksharma/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)


In [2]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option("display.max_colwidth", None)
import pandas as pd
results_mT5_XL = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/mT5_XL/model_generation.csv")
# results_mT5_XL['expected_skills'] = results_mT5_XL['expected_text'].str.findall(r'<SoftSkill>(.*?)</SoftSkill>')
results_mT5_XL = results_mT5_XL.drop(columns='Unnamed: 0', axis = 0)
results_mT5_XL = results_mT5_XL.drop(columns='generated_skills', axis = 0)
results_mT5_XL['expected_skills_all'] = results_mT5_XL['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
results_mT5_XL['generated_skills_all'] = results_mT5_XL['generated_text'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill>|/SoftSkill_C>)')
results_mT5_XL.head(4)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]"
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]"
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikations-Fähigkeit, praktisches Verständnis, manuelles Geschick]"
3,1252,Sie <SoftSkill_C>wertschätzen</SoftSkill_C> und <SoftSkill>respektieren die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill>.,Sie <SoftSkill>wertschätzen die Erfahrung der Betroffenen und deren Angehörigen</SoftSkill> und <SoftSkill>res

## GET TEST DATA 

In [3]:

import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge


# TESTING SAVED MODELS

In [4]:
import pandas as pd
import re
pd.set_option("display.max_colwidth", None)

# results_mT5_XL = pd.read_csv("/home/user/ksharma/ks_thesis/ma-thesis/bloom_peft_lora/bloom_7bn_results_ORIG_15epochs.csv")
# results_mT5_XL = results_mT5_XL[results_mT5_XL['valid'] == 1]
# results_mT5_XL = results_mT5_XL.drop(columns='Unnamed: 0', axis = 0)
# results_mT5_XL = results_mT5_XL.drop(columns='expected_skills', axis = 0)
# results_mT5_XL = results_mT5_XL.drop(columns='generated_skills', axis = 0)

# Function to extract skills between "<SoftSkill_C>" and "</SoftSkill_C>"
def extract_skills_c(text):
    return re.findall(r'<SoftSkill_C>(.*?)<\/SoftSkill_C', text)

# Function to extract skills between "<SoftSkill>"
def extract_skills(text):
    return re.findall(r'<SoftSkill>(.*?)<\/SoftSkill', text)

# Apply the functions to 'input', 'expected_text', and 'generated_text' columns
results_mT5_XL['input_skills_C'] = results_mT5_XL['input'].apply(extract_skills_c)
# df['input_skills'] = df['input'].apply(extract_skills)
results_mT5_XL['input_skills_all'] = results_mT5_XL['input'].str.findall(r'(?:<SoftSkill>|<SoftSkill_C>)(.*?)(?:</SoftSkill|</SoftSkill_C)')
# results_mT5_XL['expected_skills_all'] = results_mT5_XL['expected_text'].str.findall(r'<SoftSkill>(.*?)<\/SoftSkill')
# results_mT5_XL['generated_skills_all'] = results_mT5_XL['generated_text'].str.findall(r'SoftSkill>(.*?)/SoftSkill>')
results_mT5_XL


,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]"
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]"
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikationsfähigkeit, praktisches Verständnis, manuelles Geschick]","[psychische Gesundheit, physische Gesundheit, Kontaktfreudigkeit, gute Auffassungsgabe, Teamfähigkeit, Kommunikations-Fähigkeit, praktisches Verständnis, manue

In [5]:
results_mT5_XL[results_mT5_XL['orig_idx'] == 1910]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all
29,1910,Abgeschlossene Berufslehre als Gärtner/in\nErfahrung in einer vergleichbaren Position von Vorteil\n<SoftSkill>Freude an Verkauf</SoftSkill> und <SoftSkill_C>Kundenberatung</SoftSkill_C>\n<SoftSkill_C>Engagierte</SoftSkill_C> und <SoftSkill>belastbare Persönlichkeit</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill> und <SoftSkill>Kontaktfreudigkeit</SoftSkill,Abgeschlossene Berufslehre als Gärtner/in\nErfahrung in einer vergleichbaren Position von Vorteil\n<SoftSkill>Freude an Verkauf</SoftSkill> und <SoftSkill>Freude an Kundenberatung</SoftSkill>\n<SoftSkill>Engagierte Persönlichkeit</SoftSkill> und <SoftSkill>belastbare Persönlichkeit</SoftSkill>\n<SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill> und <SoftSkill>Kontaktfreudigkeit</SoftSkill>,MDC,"['Freude an Verkauf', 'Freude an Kundenberatung', 'Engagierte Persönlichkeit', 'belastbare Persönlichkeit', 'Ausgeprägte Teamfähigkeit', 'Kontaktfreudigkeit']",1,Abgeschlossene Berufslehre als Gärtner/in Erfahrung in einer vergleichbaren Position von Vorteil <SoftSkill>Freude an Verkauf</SoftSkill> und <SoftSkill>Freude an Kundenberatung</SoftSkill> <SoftSkill>Engagierte Persönlichkeit</SoftSkill> und <SoftSkill>belastbare Persönlichkeit</SoftSkill> <SoftSkill>Ausgeprägte Teamfähigkeit</SoftSkill> und <SoftSkill>Kontaktfreudigkeit</SoftSkill>,"[Freude an Verkauf, Freude an Kundenberatung, Engagierte Persönlichkeit, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]","[Freude an Verkauf, Freude an Kundenberatung, Engagierte Persönlichkeit, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]","[Kundenberatung, Engagierte]","[Freude an Verkauf, Kundenberatung, Engagierte, belastbare Persönlichkeit, Ausgeprägte Teamfähigkeit, Kontaktfreudigkeit]"


In [5]:
# Function to find the corresponding skills in 'expected_text' and 'generated_text'
def find_C_expected_skills(row):
    corresponding_skills = []
    for skill_c in row['input_skills_C']:
        if skill_c in row['input_skills_all']:
            idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['expected_skills_all'][idx])
            # corresponding_skills.append(row['generated_skills_new'][idx])
    return corresponding_skills

# def find_C_generated_skills(row):
#     corresponding_skills = []
#     print(row['orig_idx'])
#     # if (len(row['generated_skills_all']) == len(row['input_skills_all'])):
#     for skill_c in row['input_skills_C']:
#         if skill_c in row['input_skills_all']:
#             idx = row['input_skills_all'].index(skill_c)
#             corresponding_skills.append(row['generated_skills_all'][idx])
#             # corresponding_skills.append(row['generated_skills_new'][idx])
#     return corresponding_skills

def find_C_generated_skills(row):
    corresponding_skills = []
    # print(f"input_skills_C = {row['input_skills_C']}")
    # print(f"end=input_skills_all = {row['input_skills_all']}")
    # print(f"generated_skills_all = {row['generated_skills_all']}")
    if len(row['input_skills_all']) == len(row['generated_skills_all']):
        for skill_c in row['input_skills_C']:
            if skill_c in row['input_skills_all']:
                idx = row['input_skills_all'].index(skill_c)
            corresponding_skills.append(row['generated_skills_all'][idx])
    return corresponding_skills

# def find_C_generated_skills(row):
#     corresponding_skills = []
#     for skill_c, generated_skill in zip(row['input_skills_C'], row['generated_skills_all']):
#         if skill_c in row['input_skills_all']:
#             idx = row['input_skills_all'].index(skill_c)
#             corresponding_skills.append(generated_skill)
#     return corresponding_skills
# Apply the function to create 'corresponding_skills' column
results_mT5_XL['expected_skills_C'] = results_mT5_XL.apply(find_C_expected_skills, axis=1)
results_mT5_XL['generated_skills_C'] = results_mT5_XL.apply(find_C_generated_skills, axis=1)


# results_mT5_XL[results_mT5_XL['generated_skills_C'] == []]

In [6]:
# results_mT5_XL['num_skills_all'] = results_mT5_XL.apply(lambda row: len(row['input_skills_all']))
# results_mT5_XL['num_skills_C'] = results_mT5_XL.apply(lambda row: len(row['input_skills_C']))
results_mT5_XL['num_skills_all'] = results_mT5_XL.apply(lambda row: len(row['expected_skills_all']) if isinstance(row['expected_skills_all'], list) else 0, axis=1)
results_mT5_XL['num_skills_C'] = results_mT5_XL.apply(lambda row: len(row['expected_skills_C']) if isinstance(row['expected_skills_C'], list) else 0, axis=1)

results_mT5_XL['num_gen_skills_all'] = results_mT5_XL.apply(lambda row: len(row['generated_skills_all']) if isinstance(row['generated_skills_all'], list) else 0, axis=1)
results_mT5_XL['num_gen_skills_C'] = results_mT5_XL.apply(lambda row: len(row['generated_skills_C']) if isinstance(row['generated_skills_C'], list) else 0, axis=1)


results_mT5_XL.head(3)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C
0,752,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill_C>dienstleitungsorientierten</SoftSkill_C>, <SoftSkill_C>verantwortungsbewussten</SoftSkill_C> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.",MC,"['dienstleitungsorientierten handeln', 'verantwortungsbewussten handeln', 'teamorientierten handeln']",1,"Wir wenden uns an einen gelernten Uhrmacher mit einem <SoftSkill>dienstleitungsorientierten handeln</SoftSkill>, <SoftSkill>verantwortungsbewussten handeln</SoftSkill> wie <SoftSkill>teamorientierten handeln</SoftSkill>.","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln, teamorientierten handeln]","[dienstleitungsorientierten, verantwortungsbewussten]","[dienstleitungsorientierten, verantwortungsbewussten, teamorientierten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]","[dienstleitungsorientierten handeln, verantwortungsbewussten handeln]",3,2,3,2
1,765,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill_C>hoher Dienstleistungs-</SoftSkill_C> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,HOWA,"['Arbeitsweise ist proaktiv', 'hoher Dienstleistungsbereitschaft', 'Einsatzbereitschaft']",1,Ihre <SoftSkill>Arbeitsweise ist proaktiv</SoftSkill> mit <SoftSkill>hoher Dienstleistungsbereitschaft</SoftSkill> und <SoftSkill>Einsatzbereitschaft</SoftSkill>.,"[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]","[Arbeitsweise ist proaktiv, hoher Dienstleistungsbereitschaft, Einsatzbereitschaft]",[hoher Dienstleistungs-],"[Arbeitsweise ist proaktiv, hoher Dienstleistungs-, Einsatzbereitschaft]",[hoher Dienstleistungsbereitschaft],[hoher Dienstleistungsbereitschaft],3,1,3,1
2,1655,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill_C>psychische</SoftSkill_C> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill_C>Team-</SoftSkill_C> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikationsfähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.",HMDC,"['psychische Gesundheit', 'physische Gesundheit', 'Kontaktfreudigkeit', 'gute Auffassungsgabe', 'Teamfähigkeit', 'Kommunikationsfähigkeit', 'praktisches Verständnis', 'manuelles Geschick']",1,"Voraussetzungen sind eine abgeschlossene Volksschule (Sekundarschule A / B), <SoftSkill>psychische Gesundheit</SoftSkill> und <SoftSkill>physische Gesundheit</SoftSkill>, <SoftSkill>Kontaktfreudigkeit</SoftSkill>, eine <SoftSkill>gute Auffassungsgabe</SoftSkill>, <SoftSkill>Teamfähigkeit</SoftSkill> und <SoftSkill>Kommunikations-Fähigkeit</SoftSkill>, <SoftSkill>praktisches Verständnis</SoftSkill> und <SoftSkill>manuelles Geschick</SoftSkill>.","[

In [153]:
# results_mT5_XL.loc[results_mT5_XL['num_skills_C'] == 0, 'input'] = results_mT5_XL.loc[results_mT5_XL['num_skills_C'] == 0, 'input'].apply(lambda x: x + '>')

In [8]:
results_mT5_XL[results_mT5_XL['num_skills_C']==0]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C


In [7]:

from collections import Counter

# Create a new column 'skills_match_score'
results_mT5_XL['percent_skills_match_all'] = 0.0
results_mT5_XL['percent_skills_match_C'] = 0.0

results_mT5_XL['num_skills_match_all'] = 0.0
results_mT5_XL['num_skills_match_C'] = 0.0

# Iterate over the rows of the DataFrame
for index, row in results_mT5_XL.iterrows():
    expected_skills_all = set(row['expected_skills_all'])
    generated_skills_all = set(row['generated_skills_all'])
    
    expected_skills_C = set(row['expected_skills_C'])
    generated_skills_C = set(row['generated_skills_C'])

    count_gen_C = Counter(generated_skills_C)
    count_exp_C = Counter(expected_skills_C)

    # Find the common elements between list1 and list2
    common_elements_C = set(expected_skills_C).intersection(set(generated_skills_C))

    # Find the number of common elements, considering duplicates
    num_common_elements_C = sum(min(count_gen_C[element], count_exp_C[element]) for element in common_elements_C)

    count_gen_all = Counter(generated_skills_all)
    count_exp_all = Counter(expected_skills_all)

    # Find the common elements between list1 and list2
    common_elements_all = set(expected_skills_all).intersection(set(generated_skills_all))

    # Find the number of common elements, considering duplicates
    num_common_elements_all = sum(min(count_gen_all[element], count_exp_all[element]) for element in common_elements_all)



    # Calculate the percentage of matching skills
    if len(expected_skills_all) > 0:
        match_score_all = len(expected_skills_all.intersection(generated_skills_all)) / len(expected_skills_all) * 100
    else:
        match_score_all = 0.0
    # Calculate the percentage of matching skills
    if len(expected_skills_C) > 0:
        match_score_C = len(expected_skills_C.intersection(generated_skills_C)) / len(expected_skills_C) * 100
    else:
        match_score_C = 0.0
    
    # Assign the match score to the 'skills_match_score' column
    results_mT5_XL.at[index, 'percent_skills_match_all'] = match_score_all
    results_mT5_XL.at[index, 'percent_skills_match_C'] = match_score_C
    results_mT5_XL.at[index, 'num_skills_match_all'] = num_common_elements_all
    results_mT5_XL.at[index, 'num_skills_match_C'] = num_common_elements_C

In [10]:
results_mT5_XL[results_mT5_XL['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     53
MWA     45
MC      31
HOWA    13
MDC      8
HMDC     4
NONE     2
HMC      1
Name: count, dtype: int64

In [11]:
results_mT5_XL[results_mT5_XL['percent_skills_match_all'] == 100]['Problem type'].value_counts()

Problem type
OWA     53
MWA     45
MC      31
HOWA    13
MDC      8
HMDC     4
NONE     2
HMC      1
Name: count, dtype: int64

In [8]:
import Levenshtein
import re 
def remove_sp_char(sent):
    modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
    modified_text = re.sub(r'\n', ' ', modified_text)
    modified_text = re.sub(r'\r', ' ', modified_text)
    return modified_text

results_mT5_XL['cleaned_generated_text'] = results_mT5_XL['generated_text'].apply(remove_sp_char)
results_mT5_XL['cleaned_expected_text'] = results_mT5_XL['expected_text'].apply(remove_sp_char)
results_mT5_XL['sentence_levenshtein'] = results_mT5_XL.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
# results_mT5_XL['skills_levenshtein'] = results_mT5_XL.apply(lambda row: Levenshtein.distance(str(row['generated_skills']), str(row['expected_skills'])), axis=1)

In [13]:
results_mT5_XL[results_mT5_XL['orig_idx']==29]

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein
106,29,<SoftSkill_C>Selbständige</SoftSkill_C> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill,<SoftSkill>Selbständige Arbeitsweise</SoftSkill> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill>,OWA,"['Selbständige Arbeitsweise', 'zuverlässige Arbeitsweise']",1,<SoftSkill>Selbständige Arbeitsweise</SoftSkill> und <SoftSkill>zuverlässige Arbeitsweise</SoftSkill>,"[Selbständige Arbeitsweise, zuverlässige Arbeitsweise]","[Selbständige Arbeitsweise, zuverlässige Arbeitsweise]",[Selbständige],"[Selbständige, zuverlässige Arbeitsweise]",[Selbständige Arbeitsweise],[Selbständige Arbeitsweise],2,1,2,1,100.0,100.0,2.0,1.0,Selbständige Arbeitsweise und zuverlässige Arbeitsweise,Selbständige Arbeitsweise und zuverlässige Arbeitsweise,0


In [14]:
from rouge import Rouge
# Initialize the ROUGE scorer

rouge = Rouge()
rouge_score = rouge.get_scores("['wertschätzend', 'respektvoll miteinader kommunizieren', 'konstruktiv im Interesse']", 
                                "['wertschätzend', 'respektvoll miteinader zu kommunizieren', 'konstruktiv im Interesse']")[0]
rouge_score



{'rouge-1': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556},
 'rouge-2': {'r': 0.7142857142857143,
  'p': 0.8333333333333334,
  'f': 0.7692307642603551},
 'rouge-l': {'r': 0.875, 'p': 1.0, 'f': 0.9333333283555556}}

In [9]:
# Define functions to calculate Rouge and BLEU scores
def calculate_rouge_scores(expected_list, generated_list):
    rouge = Rouge()
    rouge_scores = []
    min_length = min(len(expected_list), len(generated_list))
    for i in range(min_length):
        expected = expected_list[i]
        generated = generated_list[i]
        scores = rouge.get_scores(expected, generated)
        rouge_scores.append(scores)
    return rouge_scores

def calculate_bleu_score(expected_list, generated_list):
    smoothing = SmoothingFunction().method0
    bleu_scores = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            bleu_score = sentence_bleu([expected], generated, smoothing_function=smoothing)
            bleu_scores.append(bleu_score)
    return bleu_scores

def calculate_levenshtein_distance(expected_list, generated_list):
    levenshtein_distances = []
    for expected, generated in zip(expected_list, generated_list):
        if len(expected_list) == len(generated_list):
            distance = Levenshtein.distance(expected, generated)
            levenshtein_distances.append(distance)
    return levenshtein_distances

# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_mT5_XL['rouge_scores_skill_C'] = results_mT5_XL.apply(lambda row: calculate_rouge_scores(row['expected_skills_C'], row['generated_skills_C']), axis=1)
results_mT5_XL['bleu_scores_skill_C'] = results_mT5_XL.apply(lambda row: calculate_bleu_score(row['expected_skills_C'], row['generated_skills_C']), axis=1)

# Calculate the average scores for each pair individually
results_mT5_XL['rouge1_avg_skill_C'] = results_mT5_XL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else-1)
results_mT5_XL['rouge2_avg_skill_C'] = results_mT5_XL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_mT5_XL['rougeL_avg_skill_C'] = results_mT5_XL['rouge_scores_skill_C'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_mT5_XL['bleu_avg_skill_C'] = results_mT5_XL['bleu_scores_skill_C'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_mT5_XL['levenshtein_skill_C'] = results_mT5_XL.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_C'], row['generated_skills_C'])), axis=1)


# Calculate the Rouge and BLEU scores for each row and store them in new columns
results_mT5_XL['rouge_scores_skill_all'] = results_mT5_XL.apply(lambda row: calculate_rouge_scores(row['expected_skills_all'], row['generated_skills_all']), axis=1)
results_mT5_XL['bleu_scores_skill_all'] = results_mT5_XL.apply(lambda row: calculate_bleu_score(row['expected_skills_all'], row['generated_skills_all']), axis=1)

# Calculate the average scores for each pair individually
results_mT5_XL['rouge1_avg_skill_all'] = results_mT5_XL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-1']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_mT5_XL['rouge2_avg_skill_all'] = results_mT5_XL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-2']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_mT5_XL['rougeL_avg_skill_all'] = results_mT5_XL['rouge_scores_skill_all'].apply(lambda x: sum(score[0]['rouge-l']['f'] for score in x) / len(x) if len(x) > 0 else -1)
results_mT5_XL['bleu_avg_skill_all'] = results_mT5_XL['bleu_scores_skill_all'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else -1)

# Calculate the Levenshtein distances for each pair individually and store the sum in a new column
results_mT5_XL['levenshtein_skill_all'] = results_mT5_XL.apply(lambda row: sum(calculate_levenshtein_distance(row['expected_skills_all'], row['generated_skills_all'])), axis=1)

results_mT5_XL[results_mT5_XL['orig_idx']==694]



,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all
5,694,"Mit Ihrer <SoftSkill_C>offenen</SoftSkill_C> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill_C>schnell</SoftSkill_C> und nach STARBUCKS® Standards.","Mit Ihrer <SoftSkill>offenen Art</SoftSkill> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill>bedienen Sie unsere Gäste schnell</SoftSkill> und nach STARBUCKS® Standards.",MDC,"['offenen Art', 'positiven Art', 'bedienen Sie unsere Gäste schnell']",1,"Mit Ihrer <SoftSkill>offenen Art</SoftSkill> und <SoftSkill>positiven Art</SoftSkill> bedienen Sie unsere Gäste freundlich, <SoftSkill>schnell bedienen Sie unsere Gäste</SoftSkill> und nach STARBUCKS® Standards.","[offenen Art, positiven Art, bedienen Sie unsere Gäste schnell]","[offenen Art, positiven Art, schnell bedienen Sie unsere Gäste]","[offenen, schnell]","[offenen, positiven Art, schnell]","[offenen Art, bedienen Sie unsere Gäste schnell]","[offenen Art, schnell bedienen Sie unsere Gäste]",3,2,3,2,66.666667,50.0,2.0,1.0,"Mit Ihrer offenen Art und positiven Art bedienen Sie unsere Gäste freundlich, schnell bedienen Sie unsere Gäste und nach STARBUCKS® Standards.","Mit Ihrer offenen Art und positiven Art bedienen Sie unsere Gäste freundlich, bedienen Sie unsere Gäste schnell und nach STARBUCKS® Standards.",17,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 0.75, 'p': 0.75, 'f': 0.749999995}, 'rouge-l': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}}]]","[1.0, 0.9255607445912747]",1.0,0.875,0.9,0.96278,16,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 0.75, 'p': 0.75, 'f': 0.749999995}, 'rouge-l': {'r': 0.8, 'p': 0.8, 'f': 0.7999999950000002}}]]","[1.0, 1.0, 0.9255607445912747]",1.0,0.916667,0.933333,0.975187,16


In [ ]:
results_mT5_XL[results_mT5_XL['orig_idx']==694]

In [16]:
results_mT5_XL.columns

Index(['orig_idx', 'input', 'expected_text', 'Problem type', 'expected_skills',
       'valid', 'generated_text', 'expected_skills_all',
       'generated_skills_all', 'input_skills_C', 'input_skills_all',
       'expected_skills_C', 'generated_skills_C', 'num_skills_all',
       'num_skills_C', 'num_gen_skills_all', 'num_gen_skills_C',
       'percent_skills_match_all', 'percent_skills_match_C',
       'num_skills_match_all', 'num_skills_match_C', 'cleaned_generated_text',
       'cleaned_expected_text', 'sentence_levenshtein', 'rouge_scores_skill_C',
       'bleu_scores_skill_C', 'rouge1_avg_skill_C', 'rouge2_avg_skill_C',
       'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C',
       'rouge_scores_skill_all', 'bleu_scores_skill_all',
       'rouge1_avg_skill_all', 'rouge2_avg_skill_all', 'rougeL_avg_skill_all',
       'bleu_avg_skill_all', 'levenshtein_skill_all'],
      dtype='object')

In [164]:
# results_mT5_XL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/bloom-7b1-lora-ORIG-stopseq20230717114608.csv")


In [17]:
results_mT5_XL['Problem type'].value_counts()

Problem type
OWA     63
MWA     53
MC      34
HOWA    19
MDC     11
HMDC    10
NONE     3
HMC      1
Name: count, dtype: int64

In [166]:
# results_mT5_XL[results_mT5_XL['skills_match_score'] == 100]['Problem type'].value_counts()

In [168]:
# import re 
# def remove_sp_char(sent):
#     modified_text = re.sub(r'(<)?(?:\/?(?:SoftSkill(?:_C)?))?(>)', '', sent)
#     modified_text = re.sub(r'\n', ' ', modified_text)
#     return modified_text

# results_mT5_XL['cleaned_generated_text'] = results_mT5_XL['generated_text'].apply(remove_sp_char)

In [18]:
# results_mT5_XL['input_C'] = results_mT5_XL.apply(lambda row: Levenshtein.distance(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)
len(results_mT5_XL[results_mT5_XL['num_skills_all'] != results_mT5_XL['num_gen_skills_all']])

6

In [10]:
pd.set_option('display.max_columns', None)  # Display all columns
# results_mT5_XL[results_mT5_XL['num_skills_C'] != results_mT5_XL['num_gen_skills_C']]
results_mT5_XL[results_mT5_XL['num_skills_all'] != results_mT5_XL['num_gen_skills_all']]
results_mT5_XL['mismatched_ALL'] = results_mT5_XL['num_skills_all'] != results_mT5_XL['num_gen_skills_all']
results_mT5_XL[results_mT5_XL['num_skills_all'] != results_mT5_XL['num_gen_skills_all']].head(3)

,orig_idx,input,expected_text,Problem type,expected_skills,valid,generated_text,expected_skills_all,generated_skills_all,input_skills_C,input_skills_all,expected_skills_C,generated_skills_C,num_skills_all,num_skills_C,num_gen_skills_all,num_gen_skills_C,percent_skills_match_all,percent_skills_match_C,num_skills_match_all,num_skills_match_C,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,rouge_scores_skill_C,bleu_scores_skill_C,rouge1_avg_skill_C,rouge2_avg_skill_C,rougeL_avg_skill_C,bleu_avg_skill_C,levenshtein_skill_C,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL
52,990,"• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill_C>Mode</SoftSkill_C>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• <SoftSkill>Interesse am Detailhandel</SoftSkill> und <SoftSkill>Interesse an der Mode</SoftSkill>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.",MWA,"['Interesse am Detailhandel', 'Interesse an der Mode', 'motivierte Persönlichkeiten', 'gerne im Team arbeiten']",1,"• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill>Mode</SoftSkill> Wir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst sind</SoftSkill>.","[Interesse am Detailhandel, Interesse an der Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten, verantwortungsbewusst sind]",[Mode],"[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]",[Interesse an der Mode],[],4,1,5,0,75.000000,0.0,3.0,0.0,"• Interesse am Detailhandel und der Mode Wir wenden uns an junge, motivierte Persönlichkeiten, die gerne im Team arbeiten und verantwortungsbewusst sind.","• Interesse am Detailhandel und Interesse an der Mode Wir wenden uns an junge, motivierte Persönlichkeiten, die gerne im Team arbeiten und verantwortungsbewusst sind.",14,[],[],-1.00,-1.0,-1.00,-1.00000,0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],0.850000,0.750000,0.850000,-1.0,0,True
60,507,Sie <SoftSkill_C>denken</SoftSkill_C> und <SoftSkill>handeln kunden-</SoftSkill> und <SoftSkill_C>qualitätsorientiert</SoftSkill_C>.,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>denken qualitätsorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,HMDC,"['denken kundenorientiert', 'handeln kundenorientiert', 'denken qualitätsorientiert', 'handeln qualitätsorientiert']",1,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,"[denken kundenorientiert, handeln kundenorientiert, denken qualitätsorientiert, handeln qualitätsorientiert]","[denken kundenorientiert, handeln kundenorientiert, handeln qualitätsorientiert]","[denken, qualitätsorientiert]","[denken, handeln kunden-, qualitätsorientiert]

In [11]:
results_C = results_mT5_XL[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_C', 'expected_skills_C', 'generated_skills_C',
                'num_skills_C', 'num_gen_skills_C', 'percent_skills_match_C', 'num_skills_match_C',
                'rouge_scores_skill_C', 'bleu_scores_skill_C', 'rouge1_avg_skill_C',
                'rouge2_avg_skill_C', 'rougeL_avg_skill_C', 'bleu_avg_skill_C', 'levenshtein_skill_C', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_ALL" DataFrame
results_ALL = results_mT5_XL[['orig_idx', 'input', 'expected_text', 'generated_text', 'input_skills_all', 'expected_skills_all', 'generated_skills_all',
                  'num_skills_all', 'num_gen_skills_all', 'percent_skills_match_all', 'num_skills_match_all',
                  'rouge_scores_skill_all', 'bleu_scores_skill_all', 'rouge1_avg_skill_all',
                  'rouge2_avg_skill_all', 'rougeL_avg_skill_all', 'bleu_avg_skill_all', 'levenshtein_skill_all', 'mismatched_ALL','Problem type',
                'valid']]

# Create "results_SENTENCE" DataFrame
results_SENTENCE = results_mT5_XL[['orig_idx', 'input', 'expected_text', 'generated_text', 'cleaned_generated_text', 'cleaned_expected_text', 'sentence_levenshtein','Problem type',
                'valid']]

# # Export the three DataFrames to separate CSV files (optional)
# results_C.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_C.csv", index=False)
# results_ALL.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_ALL.csv", index=False)
# results_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/BLOOM_results_SENTENCE.csv", index=False)

In [21]:
results_ALL[results_ALL['mismatched_ALL']==True].head(5)

,orig_idx,input,expected_text,generated_text,input_skills_all,expected_skills_all,generated_skills_all,num_skills_all,num_gen_skills_all,percent_skills_match_all,num_skills_match_all,rouge_scores_skill_all,bleu_scores_skill_all,rouge1_avg_skill_all,rouge2_avg_skill_all,rougeL_avg_skill_all,bleu_avg_skill_all,levenshtein_skill_all,mismatched_ALL,Problem type,valid
52,990,"• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill_C>Mode</SoftSkill_C>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• <SoftSkill>Interesse am Detailhandel</SoftSkill> und <SoftSkill>Interesse an der Mode</SoftSkill>\nWir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst\nsind</SoftSkill>.","• <SoftSkill>Interesse am Detailhandel</SoftSkill> und der <SoftSkill>Mode</SoftSkill> Wir wenden uns an junge, <SoftSkill>motivierte Persönlichkeiten</SoftSkill>, die <SoftSkill>gerne im Team arbeiten</SoftSkill> und <SoftSkill>verantwortungsbewusst sind</SoftSkill>.","[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Interesse an der Mode, motivierte Persönlichkeiten, gerne im Team arbeiten]","[Interesse am Detailhandel, Mode, motivierte Persönlichkeiten, gerne im Team arbeiten, verantwortungsbewusst sind]",4,5,75.000000,3.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.25, 'f': 0.39999999680000003}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]]",[],0.850000,0.750000,0.850000,-1.0,0,True,MWA,1
60,507,Sie <SoftSkill_C>denken</SoftSkill_C> und <SoftSkill>handeln kunden-</SoftSkill> und <SoftSkill_C>qualitätsorientiert</SoftSkill_C>.,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>denken qualitätsorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,Sie <SoftSkill>denken kundenorientiert</SoftSkill> und <SoftSkill>handeln kundenorientiert</SoftSkill> und <SoftSkill>handeln qualitätsorientiert</SoftSkill>.,"[denken, handeln kunden-, qualitätsorientiert]","[denken kundenorientiert, handeln kundenorientiert, denken qualitätsorientiert, handeln qualitätsorientiert]","[denken kundenorientiert, handeln kundenorientiert, handeln qualitätsorientiert]",4,3,75.000000,3.0,"[[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}], [{'rouge-1': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.5, 'p': 0.5, 'f': 0.4999999950000001}}]]",[],0.833333,0.666667,0.833333,-1.0,0,True,HMDC,1
117,135,"Ihre <SoftSkill>Begeisterung für die Marken Jaeger-LeCoultre und Tissot</SoftSkill> sowie Ihr <SoftSkill_C>stilvoller</SoftSkill_C>, <SoftSkill>souveräner Umgang auch in hektischen Situationen prägen Ihre Persönlichkeit</SoftSkill>.","Ihre <SoftSkill>Begeisterung für die Marken Jaeger-LeCoultre und Tissot</SoftSkill> sowie Ihr <SoftSkill>stilvoller Umgang</SoftSkill>, <SoftSkill>souveräner Umgang auch in hektischen Situationen prägen Ihre Persönlichkeit</Soft

In [22]:
pd.set_option('display.max_row', None)
results_C['levenshtein_skill_C'].mean()

1.8

## Sentence ROUGE

In [12]:
from rouge import Rouge

# Function to calculate Rouge scores
def calculate_rouge_scores(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(reference, candidate)
    rouge_1_f = scores[0]['rouge-1']['f'] if 'rouge-1' in scores[0] else None
    rouge_2_f = scores[0]['rouge-2']['f'] if 'rouge-2' in scores[0] else None
    rouge_l_f = scores[0]['rouge-l']['f'] if 'rouge-l' in scores[0] else None
    return rouge_1_f, rouge_2_f, rouge_l_f

# Apply the functions to calculate the scores and add them as new columns
results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
    lambda row: calculate_rouge_scores(row['cleaned_expected_text'], row['cleaned_generated_text']),
    axis=1
))

/tmp/ipykernel_927392/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_927392/809110196.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['rouge1_score'], results_SENTENCE['rouge2_score'], results_SENTENCE['rougeL_score'] = zip(*results_SENTENCE.apply(
/tmp/ipykernel_927392/809110196.py:13: SettingWithCopyWarning: 
A value is try

## Sentence BERT Cosine Similarity

In [13]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch

# Assuming you have already read the DataFrame from the CSV file
# results_SENTENCE = pd.read_csv("/path/to/results_SENTENCE.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to calculate cosine similarity
def calculate_cosine_similarity(sentence1, sentence2):
    # Tokenize and convert sentences to tensors
    inputs = tokenizer([sentence1, sentence2], return_tensors="pt", padding=True, truncation=True)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    # Normalize embeddings
    normalized_embeddings = embeddings / (embeddings ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity
    similarity = (normalized_embeddings[0] @ normalized_embeddings[1].T).item()
    return similarity

# Apply the function to calculate cosine similarity and add it as a new column
results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


/tmp/ipykernel_927392/2919956664.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_SENTENCE['cosine_similarity'] = results_SENTENCE.apply(lambda row: calculate_cosine_similarity(row['cleaned_generated_text'], row['cleaned_expected_text']), axis=1)


In [76]:
# results_SENTENCE[(results_SENTENCE['sentence_levenshtein'] > 3) & (results_SENTENCE['sentence_levenshtein'] < 6)].tail(20)
results_SENTENCE[(results_SENTENCE['orig_idx'] == 237)]

,orig_idx,input,expected_text,generated_text,cleaned_generated_text,cleaned_expected_text,sentence_levenshtein,Problem type,valid,rouge1_score,rouge2_score,rougeL_score,cosine_similarity
144,237,Zum Beruf gehören auch ein <SoftSkill_C>grosses\n</SoftSkill_C><SoftSkill_C>Einfühlungs-</SoftSkill_C> und <SoftSkill>Beobachtungsvermögen</SoftSkill> ebenso wie die <SoftSkill>Bereitschaft das eigene Handeln\nzu reflektieren</SoftSkill> und ein <SoftSkill>ausgeprägtes Verantwortungsbewusstsein</SoftSkill>.,Zum Beruf gehören auch ein <SoftSkill>grosses Einfühlungsvermögen</SoftSkill> und <SoftSkill>grosses Beobachtungsvermögen</SoftSkill> ebenso wie die <SoftSkill>Bereitschaft das eigene Handeln zu reflektieren</SoftSkill> und ein <SoftSkill>ausgeprägtes Verantwortungsbewusstsein</SoftSkill>.,Zum Beruf gehören auch ein <SoftSkill>grosses Einfühlungsvermögen</SoftSkill> und <SoftSkill>Beobachtungsvermögen</SoftSkill> ebenso wie die <SoftSkill>Bereitschaft das eigene Handeln zu reflektieren</SoftSkill> und ein <SoftSkill>ausgeprägtes Verantwortungsbewusstsein</SoftSkill>.,Zum Beruf gehören auch ein grosses Einfühlungsvermögen und Beobachtungsvermögen ebenso wie die Bereitschaft das eigene Handeln zu reflektieren und ein ausgeprägtes Verantwortungsbewusstsein.,Zum Beruf gehören auch ein grosses Einfühlungsvermögen und grosses Beobachtungsvermögen ebenso wie die Bereitschaft das eigene Handeln zu reflektieren und ein ausgeprägtes Verantwortungsbewusstsein.,8,HOWA,1,1.0,0.930233,1.0,0.999263


## FUnction for cosine similarity for _C and ALL

In [14]:

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings for a list of sentences
def get_bert_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate cosine similarity between two lists of sentences
def calculate_cosine_similarity(list1, list2):
    if not list1 or not list2:
        return np.nan

    # Get BERT embeddings for both lists of sentences
    embeddings1 = get_bert_embeddings(list1)
    embeddings2 = get_bert_embeddings(list2)

    # Normalize embeddings
    normalized_embeddings1 = embeddings1 / (embeddings1 ** 2).sum(axis=-1, keepdims=True) ** 0.5
    normalized_embeddings2 = embeddings2 / (embeddings2 ** 2).sum(axis=-1, keepdims=True) ** 0.5

    # Calculate cosine similarity for each sentence pair
    similarity_values = np.dot(normalized_embeddings1, normalized_embeddings2.T)
    return similarity_values.diagonal().tolist()


In [15]:

# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_C.iterrows():
    list1 = row['expected_skills_C']
    list2 = row['generated_skills_C']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_C['pairwise_cosine_similarity'] = cosine_similarity_list



# Apply the function to calculate pairwise cosine similarity and store the distances in a list
cosine_similarity_list = []
for index, row in results_ALL.iterrows():
    list1 = row['expected_skills_all']
    list2 = row['generated_skills_all']

    # Calculate cosine similarity for each sentence pair in both lists
    pairwise_similarities = calculate_cosine_similarity(list1, list2)

    # Append the list of pairwise similarities for the row
    cosine_similarity_list.append(pairwise_similarities)

# Add the list of pairwise cosine similarity distances as a new column in the DataFrame
results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list

results_C['average_cosine_similarity'] = results_C['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)
results_ALL['average_cosine_similarity'] = results_ALL['pairwise_cosine_similarity'].apply(lambda x: np.mean(x) if x else np.nan)


/tmp/ipykernel_927392/2476177218.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_C['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_927392/2476177218.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_ALL['pairwise_cosine_similarity'] = cosine_similarity_list
/tmp/ipykernel_927392/2476177218.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [157]:
# results_ALL[results_ALL['mismatched_ALL']==True].head(5)

In [28]:
results_SENTENCE.columns

Index(['orig_idx', 'input', 'expected_text', 'generated_text',
       'cleaned_generated_text', 'cleaned_expected_text',
       'sentence_levenshtein', 'Problem type', 'valid', 'rouge1_score',
       'rouge2_score', 'rougeL_score', 'cosine_similarity'],
      dtype='object')

## Prepare Metrics and Tables 

In [29]:
# Create a DataFrame for _C metrics
summary_C_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (C)', 'Average Rouge-2 Score (C)',
               'Average Rouge-L Score (C)',  'Average Levenshtein (C)',
               'Weighted Avg. Percentage Skills (C)', 'Cosine Similarity(C)'],
    'Value': [results_C['rouge1_avg_skill_C'].mean(), results_C['rouge2_avg_skill_C'].mean(),
              results_C['rougeL_avg_skill_C'].mean(),results_C['levenshtein_skill_C'].mean(),
              results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),results_C['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_ALL_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (ALL)', 'Average Rouge-2 Score (ALL)',
               'Average Rouge-L Score (ALL)', 'Average Levenshtein (ALL)',
               'Weighted Avg. Percentage Skills (ALL)', 'Cosine Similarity(ALL)'],
    'Value': [results_ALL['rouge1_avg_skill_all'].mean(), results_ALL['rouge2_avg_skill_all'].mean(),
              results_ALL['rougeL_avg_skill_all'].mean(), results_ALL['levenshtein_skill_all'].mean(),
              results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),results_ALL['average_cosine_similarity'].mean()]
})

# Create a DataFrame for "ALL" metrics
summary_SENTENCE_df = pd.DataFrame({
    'Metric': ['Average Rouge-1 Score (SENTENCE)', 'Average Rouge-2 Score (SENTENCE)',
               'Average Rouge-L Score (SENTENCE)', 'Average Levenshtein (SENTENCE)',
               'Cosine Similarity(SENTENCE)'],
    'Value': [results_SENTENCE['rouge1_score'].mean(), results_SENTENCE['rouge2_score'].mean(),
              results_SENTENCE['rougeL_score'].mean(), results_SENTENCE['sentence_levenshtein'].mean(),
              results_SENTENCE['cosine_similarity'].mean()]
})

# Export the summary DataFrames to CSV
# summary_C_df.to_csv("summary_metrics_C.csv", index=False)
# summary_ALL_df.to_csv("summary_metrics_ALL.csv", index=False)

In [30]:
summary_C_df

,Metric,Value
0,Average Rouge-1 Score (C),0.879583
1,Average Rouge-2 Score (C),0.802188
2,Average Rouge-L Score (C),0.879070
3,Average Levenshtein (C),1.800000
4,Weighted Avg. Percentage Skills (C),0.850187
5,Cosine Similarity(C),0.994656


In [34]:
summary_C_df


,Metric,Value
0,Average Rouge-1 Score (C),0.604258
1,Average Rouge-2 Score (C),0.507258
2,Average Rouge-L Score (C),0.604258
3,Average Levenshtein (C),15.835897
4,Weighted Avg. Percentage Skills (C),0.520599
5,Cosine Similarity(C),0.958183


In [35]:
summary_ALL_df

,Metric,Value
0,Average Rouge-1 Score (ALL),0.962526
1,Average Rouge-2 Score (ALL),0.843614
2,Average Rouge-L Score (ALL),0.962184
3,Average Levenshtein (ALL),2.179487
4,Weighted Avg. Percentage Skills (ALL),0.925811
5,Cosine Similarity(ALL),0.995907


In [36]:
summary_SENTENCE_df

,Metric,Value
0,Average Rouge-1 Score (SENTENCE),0.981805
1,Average Rouge-2 Score (SENTENCE),0.958967
2,Average Rouge-L Score (SENTENCE),0.981463
3,Average Levenshtein (SENTENCE),5.133333
4,Cosine Similarity(SENTENCE),0.991043


## Problem Types DATA RESULTS _C SKILLS

In [31]:
summary_C_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_C = results_C.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_C:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_C_df = pd.DataFrame({
        'Average Rouge-1 Score (C)': [group['rouge1_avg_skill_C'].mean()],
        'Average Rouge-2 Score (C)': [group['rouge2_avg_skill_C'].mean()],
        'Average Rouge-L Score (C)': [group['rougeL_avg_skill_C'].mean()],
        'Average Levenshtein (C)': [group['levenshtein_skill_C'].mean()],
        'Weighted Avg. Percentage Skills (C)': [group['num_skills_match_C'].sum() / group['num_skills_C'].sum()],
        'Cosine Similarity (C)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_C_df_list.append(summary_C_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_df = pd.concat(summary_C_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages = {
    'Average Rouge-1 Score (C)': results_C['rouge1_avg_skill_C'].mean(),
    'Average Rouge-2 Score (C)': results_C['rouge2_avg_skill_C'].mean(),
    'Average Rouge-L Score (C)': results_C['rougeL_avg_skill_C'].mean(),
    'Average Levenshtein (C)': results_C['levenshtein_skill_C'].mean(),
    'Weighted Avg. Percentage Skills (C)': results_C['num_skills_match_C'].sum() / results_C['num_skills_C'].sum(),
    'Cosine Similarity (C)': results_C['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_C)
}

overall_summary_df = pd.DataFrame(overall_averages, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_df
result_summary_df_C_skills = pd.concat([result_summary_df, overall_summary_df])

result_summary_df_C_skills['model'] = 'mT5_XL'
result_summary_df_C_skills


,Average Rouge-1 Score (C),Average Rouge-2 Score (C),Average Rouge-L Score (C),Average Levenshtein (C),Weighted Avg. Percentage Skills (C),Cosine Similarity (C),Number of Samples,model
HMC,1.000000,0.500000,1.000000,0.000000,1.000000,1.000000,1,mT5_XL
HMDC,0.891667,0.683333,0.891667,4.400000,0.840000,0.984865,10,mT5_XL
HOWA,0.789474,0.526316,0.789474,1.052632,0.789474,0.989858,19,mT5_XL
MC,0.919499,0.887955,0.919499,2.294118,0.904110,0.998409,34,mT5_XL
MDC,0.772727,0.715909,0.763636,3.272727,0.782609,0.993108,11,mT5_XL
MWA,0.891826,0.862159,0.891826,1.792453,0.821429,0.995088,53,mT5_XL
NONE,0.740741,0.333333,0.740741,12.333333,0.666667,0.963408,3,mT5_XL
OWA,0.894444,0.862434,0.894444,0.650794,0.859375,0.996901,63,mT5_XL
All Samples,0.879583,0.802188,0.879070,1.800000,0.850187,0.994656,195,mT5_XL


In [32]:

result_summary_df_C_skills['Problem Type'] = result_summary_df_C_skills.index
new_df = result_summary_df_C_skills.iloc[:, [2, 3, 4, 5, 8]]
latex_table = new_df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrrrl}
\toprule
Average Rouge-L Score (C) & Average Levenshtein (C) & Weighted Avg. Percentage Skills (C) & Cosine Similarity (C) & Problem Type \\
\midrule
1.000000 & 0.000000 & 1.000000 & 1.000000 & HMC \\
0.891667 & 4.400000 & 0.840000 & 0.984865 & HMDC \\
0.789474 & 1.052632 & 0.789474 & 0.989858 & HOWA \\
0.919499 & 2.294118 & 0.904110 & 0.998409 & MC \\
0.763636 & 3.272727 & 0.782609 & 0.993108 & MDC \\
0.891826 & 1.792453 & 0.821429 & 0.995088 & MWA \\
0.740741 & 12.333333 & 0.666667 & 0.963408 & NONE \\
0.894444 & 0.650794 & 0.859375 & 0.996901 & OWA \\
0.879070 & 1.800000 & 0.850187 & 0.994656 & All Samples \\
\bottomrule
\end{tabular}



## Problem Types DATA RESULTS ALL SKILLS

In [38]:
# Create an empty list to store the summary DataFrames for each category
summary_ALL_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_ALL = results_ALL.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_ALL:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_ALL_df = pd.DataFrame({
        'Average Rouge-1 Score (ALL)': [group['rouge1_avg_skill_all'].mean()],
        'Average Rouge-2 Score (ALL)': [group['rouge2_avg_skill_all'].mean()],
        'Average Rouge-L Score (ALL)': [group['rougeL_avg_skill_all'].mean()],
        'Average Levenshtein (ALL)': [group['levenshtein_skill_all'].mean()],
        'Weighted Avg. Percentage Skills (ALL)': [group['num_skills_match_all'].sum() / group['num_skills_all'].sum()],
        'Cosine Similarity (ALL)': [group['average_cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_ALL_df_list.append(summary_ALL_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_ALL_df = pd.concat(summary_ALL_df_list)

# Calculate overall averages across all categories and append as a new row
overall_averages_ALL = {
    'Average Rouge-1 Score (ALL)': results_ALL['rouge1_avg_skill_all'].mean(),
    'Average Rouge-2 Score (ALL)': results_ALL['rouge2_avg_skill_all'].mean(),
    'Average Rouge-L Score (ALL)': results_ALL['rougeL_avg_skill_all'].mean(),
    'Average Levenshtein (ALL)': results_ALL['levenshtein_skill_all'].mean(),
    'Weighted Avg. Percentage Skills (ALL)': results_ALL['num_skills_match_all'].sum() / results_ALL['num_skills_all'].sum(),
    'Cosine Similarity (ALL)': results_ALL['average_cosine_similarity'].mean(),
    'Number of Samples': len(results_ALL)
}

overall_summary_ALL_df = pd.DataFrame(overall_averages_ALL, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_ALL_df
result_summary_df_ALL_skills = pd.concat([result_summary_ALL_df, overall_summary_ALL_df])

result_summary_df_ALL_skills['model'] = 'mT5_XL'
result_summary_df_ALL_skills


,Average Rouge-1 Score (ALL),Average Rouge-2 Score (ALL),Average Rouge-L Score (ALL),Average Levenshtein (ALL),Weighted Avg. Percentage Skills (ALL),Cosine Similarity (ALL),Number of Samples,model
HMC,1.000000,0.333333,1.000000,0.000000,1.000000,1.000000,1,mT5_XL
HMDC,0.905000,0.694167,0.905000,4.300000,0.882353,0.990551,10,mT5_XL
HOWA,0.948977,0.518421,0.948977,2.894737,0.888889,0.996322,19,mT5_XL
MC,0.987190,0.956466,0.987190,2.294118,0.962406,0.999150,34,mT5_XL
MDC,0.900253,0.816116,0.894192,5.090909,0.872340,0.986450,11,mT5_XL
MWA,0.970806,0.889266,0.970806,1.792453,0.936306,0.995910,53,mT5_XL
NONE,0.870370,0.666667,0.870370,12.333333,0.833333,0.981704,3,mT5_XL
OWA,0.969538,0.895528,0.969538,0.968254,0.929348,0.997076,63,mT5_XL
All Samples,0.962526,0.843614,0.962184,2.179487,0.925811,0.995907,195,mT5_XL


## Problem Types DATA RESULTS SENTENCE SKILLS

In [39]:
# Create an empty list to store the summary DataFrames for each category
summary_SENTENCE_df_list = []

# Group the DataFrame by "Problem Type"
grouped_results_SENTENCE = results_SENTENCE.groupby('Problem type')

# Iterate over each group and calculate the summary DataFrame
for problem_type, group in grouped_results_SENTENCE:
    num_samples = len(group)  # Calculate the number of samples for the current category
    summary_SENTENCE_df = pd.DataFrame({
        'Average Rouge-1 Score (SENTENCE)': [group['rouge1_score'].mean()],
        'Average Rouge-2 Score (SENTENCE)': [group['rouge2_score'].mean()],
        'Average Rouge-L Score (SENTENCE)': [group['rougeL_score'].mean()],
        'Average Levenshtein (SENTENCE)': [group['sentence_levenshtein'].mean()],
        'Cosine Similarity (SENTENCE)': [group['cosine_similarity'].mean()],
        'Number of Samples': [num_samples]
    }, index=[problem_type])
    # Add the summary DataFrame for the current category to the list
    summary_SENTENCE_df_list.append(summary_SENTENCE_df)

# Concatenate the summary DataFrames into a single DataFrame
result_summary_SENTENCE_df = pd.concat(summary_SENTENCE_df_list)

# Calculate overall averages across all categories and add as a new row
overall_averages_SENTENCE = {
    'Average Rouge-1 Score (SENTENCE)': results_SENTENCE['rouge1_score'].mean(),
    'Average Rouge-2 Score (SENTENCE)': results_SENTENCE['rouge2_score'].mean(),
    'Average Rouge-L Score (SENTENCE)': results_SENTENCE['rougeL_score'].mean(),
    'Average Levenshtein (SENTENCE)': results_SENTENCE['sentence_levenshtein'].mean(),
    'Cosine Similarity (SENTENCE)': results_SENTENCE['cosine_similarity'].mean(),
    'Number of Samples': len(results_SENTENCE)
}

overall_summary_SENTENCE_df = pd.DataFrame(overall_averages_SENTENCE, index=['All Samples'])

# Concatenate the overall summary as a new row to the result_summary_SENTENCE_df
result_summary_df_SENTENCE = pd.concat([result_summary_SENTENCE_df, overall_summary_SENTENCE_df])

result_summary_df_SENTENCE['model'] = 'mT5_XL'
result_summary_df_SENTENCE

,Average Rouge-1 Score (SENTENCE),Average Rouge-2 Score (SENTENCE),Average Rouge-L Score (SENTENCE),Average Levenshtein (SENTENCE),Cosine Similarity (SENTENCE),Number of Samples,model
HMC,1.000000,1.000000,1.000000,1.000000,1.000000,1,mT5_XL
HMDC,0.968930,0.916381,0.968930,8.500000,0.995142,10,mT5_XL
HOWA,0.983736,0.952359,0.983736,3.368421,0.992810,19,mT5_XL
MC,0.994611,0.984012,0.994611,3.558824,0.996632,34,mT5_XL
MDC,0.989091,0.948992,0.983030,4.000000,0.997100,11,mT5_XL
MWA,0.980444,0.957569,0.980444,3.264151,0.987359,53,mT5_XL
NONE,1.000000,0.956522,1.000000,12.333333,0.990597,3,mT5_XL
OWA,0.974783,0.955934,0.974783,7.539683,0.988621,63,mT5_XL
All Samples,0.981805,0.958967,0.981463,5.133333,0.991043,195,mT5_XL


In [40]:
result_summary_df_C_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/mT5_XL/summary_metrics_C_mT5_XL.csv", index=False)
result_summary_df_ALL_skills.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/mT5_XL/summary_metrics_ALL_mT5_XL.csv", index=False)
result_summary_df_SENTENCE.to_csv("/home/user/ksharma/ks_thesis/ma-thesis/result_evaluation/data_files/mT5_XL/summary_metrics_SENTENCE_mT5_XL.csv", index=False)